In [5]:
import numpy as np
import pandas as pd
import MySQLdb
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Define all models 

def engelandmay(pre_transport, boulder_density, Vabc, abc):
    q = 0.73 # boulder area coefficient. empirically determined average value (Aphoto/Aaxes)
    
    if pre_transport == "Joint Bounded":
        # Minimum wave height of a tsunami to dislodge a boulder in JBB scenario
        H_T_dislodge = ((boulder_density-water_density)*Vabc**np.cos(alpha)+miu*np.sin(alpha))/(2*water_density*c_l*abc[0]*abc[1]*q)

        # Minimum swell wave height to dislodge a boulder in JBB scenario
        H_S_dislodge = ((boulder_density-water_density)*Vabc**np.cos(alpha)+miu*np.sin(alpha))/(0.5*water_density*c_l*abc[0]*abc[1]*q)
        
    else:
        H_T_dislodge = None
        H_S_dislodge = None

    # Minimum wave height for storm waves to deposit boulder at location
    H_S_deposit = (2*miu*Vabc*boulder_density)/(c_d*abc[0]*abc[2]*q*water_density)
    
    # Minimum wave height for tsunami waves to deposit boulder at location
    H_T_deposit = (0.5*miu*Vabc*boulder_density)/(c_d*abc[0]*abc[2]*q*water_density)
    
    return(H_T_dislodge, H_S_dislodge, H_S_deposit, H_T_deposit)

def nandasena(pre_transport, boulder_density, abc):
    # Minimum flow velocity for incipient motion for submerged and subaerial boulders 
    # Assumed that in subaerial setting, the flow acceleration is insignificant, so inertia force can be ignored. 
    if pre_transport == "Submerged" or pre_transport == "Subaerial":
        
        sliding_condition = (2*(boulder_density/water_density-1)*g*abc[2]*(miu*np.cos(alpha)+np.sin(alpha)))/(c_d*(abc[2]/abc[1])+miu_s*c_l)
        overturn_rolling_condition = (2*(boulder_density/water_density-1)*g*abc[2]*(np.cos(alpha)+(abc[2]/abc[1])*np.sin(alpha)))/((c_d)*(abc[2]**2/abc[1]**2)+c_l)
        saltation_condition = (2*(boulder_density/water_density-1)*g*abc[2]*np.cos(alpha))/c_l
        
        u_square = min(sliding_condition, overturn_rolling_condition, saltation_condition)
        
        if u_square == sliding_condition:
            transport_mode = "Sliding"
        
        elif u_square == overturn_rolling_condition:
            transport_mode = "Overturn or Rolling"
        
        elif u_square == saltation_condition:
            transport_mode = "Saltation"
        
        u = u_square**0.5
    
    # Minimum flow velocity for incipient motion for joint bounded boulders
    elif pre_transport == "Joint Bounded":
        u = ((2*(boulder_density/water_density-1)*g*abc[2]*(np.cos(alpha)+miu_s*np.sin(alpha)))/c_l)**0.5
        transport_mode = None
    
    else:
        u = None
        transport_mode = None
        
    return (u, transport_mode)

def pignatelli(boulder_density, abc):
    # Minimum tsunami wave height for incipient motion (H_T)                        
    H_T = abc[2]*(boulder_density/water_density-1)/(2*c_l)
 
    # Transport distance of boulder inland (D)
    k = 0.06                    # constant that eqals 0.06 for most tsunamis
    if H_T > h_c: 
        D = X_max-((H_T-h_c)**1.33)*(manning**(-2))*k*np.cos(alpha)
    else:
        D = 0
    
    return (H_T, D)

# Manning table
manning_dict = {"Lagoon, fluvial plain": (0.01, 0.015),
                "Mediterranean vegetation": (0.016, 0.025),
                "Farm area": (0.026, 0.035),
                "Discontinuous dune belts (without vegetation)": (0.036, 0.040),
                "Dune belts (AltitudeN3 m)": (0.041, 0.046),
                "Rocky coasts (very karsificated)": (0.047, 0.052),
                "Urban area discontinuous": (0.053, 0.058),
                "Urban area (with buildings very concentrated)": (0.059, 0.064),
                "Mangroves": (0.065, 0.069),
                "Forests, Pinewood, etc.": (0.07, 0.07)
                }

def goto_apply(boulder_density, Vabc, abc, delta):
    # Let y be the speed of the boulder
    # Implement assuming that wave height is approximately a constant value
    def goto_model(y,t,H,v_miu):
        Fb = v_miu*(boulder_density-water_density)*Vabc*g*np.cos(y/abs(y))
        Fg = (boulder_density-water_density)*Vabc*g*np.sin(alpha)
        dydt=(0.5*c_d*water_density*abc[0]*abc[2]*(np.sqrt(delta*g*H)-y)*abs(np.sqrt(delta*g*H)-y)-Fb-Fg)/(boulder_density*Vabc+(c_m-1)*water_density*Vabc)
        return dydt

    # Initial Conditions
    y0=0.0000001

    # Time scale to solve ode
    t = np.linspace(0,1000, 200)

    # Solve ODE for y
    y = odeint(goto_model, y0, t, args=(H,v_miu))

    # Plot results
    #plt.plot(t,y)
    #plt.xlabel('Time')
    #plt.ylabel('Speed of the Boulder')
    #plt.show
    return y[95][0] # Return one of the speed values at a later time point

def rms(predicted, actual): # Generates the RMS value per model
    RMS = 0
    flag = False
    for l in range(len(predicted)):
        p = predicted[l]
        a = actual[l]
        if (a!=None) and (p!=None): # only sum the RMS for boulders that have the relevant data
            sq_diff = (p-a)**2
            RMS = RMS + sq_diff
            flag = True
    if flag==True:        
        return(RMS**0.5) 
    
    else:
        return(None)

In [16]:
# Run the models

def runmodels():
    emls = []
    gls = []
    nls = []
    pls = []
    lt_actual = []


    for id, row in filt_df.iterrows():
        # Assign variable names to desired variables
        boulder_ID = filt_df.loc[id, 5]
        pre_transport = filt_df.loc[id, 21]
        boulder_density = filt_df.loc[id, 14]
        Vabc = filt_df.loc[id, 9]
        abc = (filt_df.loc[id, 6], filt_df.loc[id, 7], filt_df.loc[id, 8])
        lateral_transport_distance = filt_df.loc[id, 22]

        # Run the Engel and May Model
        resultem = engelandmay(pre_transport, boulder_density, Vabc, abc)
        emls.append(resultem)
        #print("Engel and May", resultem) # H_T_dislodge, H_S_dislodge, H_S_deposit, H_T_deposit

        # Run the Goto Model
        resultgoto_tsunami = goto_apply(boulder_density, Vabc, abc, 4) # delta = 4 Wave typology for Tsunami from Fukui et. al 1973
        resultgoto_storm = goto_apply(boulder_density, Vabc, abc, 1) # delta = 1 Wave typology for Swell Wave from Fukui et. al 1973
        gls.append((resultgoto_tsunami, resultgoto_storm))
        #print("Goto", resultgoto_tsunami, resultgoto_storm) # Boulder speed

        # Run the models that require a pre-transport setting
        if pre_transport != None: 

            # Run the Nandansena model
            resultn = nandasena(pre_transport, boulder_density, abc)
            nls.append(resultn)
            #print("Nandasena", resultn) # Minimum velocity

            # Run the Pignatelli model
            if pre_transport == "Joint Bounded":
                resultp = pignatelli(boulder_density, abc)
                pls.append((resultp[0], resultp[1]))
                #print("Pignatelli", resultp)  #H_T, D
                
            else:
                pls.append((None, None))
                #print("Pignatelli", (None, None))
        
        else:
            pls.append((None, None))
            #nls.append(None)

        if lateral_transport_distance != None:
            lt_actual.append(lateral_transport_distance)

        else:
            lt_actual.append(None)
        
    #print("Engel and May:", len(emls), "\t", "Goto:", len(gls), "\t", "Nandasena:", len(nls), "\t", "Pignatelli:", len(pls))
    return(emls, gls, nls, pls, lt_actual)

In [1]:
# PLots of interest

def interestplots(emls, gls, nls, pls, pls_actual):
    # Density plots for Engel and May
    HT_dislodge = []
    HS_dislodge = []
    HS_deposit = []
    HT_deposit = []
    nls_u = []
    nls_transport = []
    gls_s = []
    gls_t = []

    for em in range(len(emls)):
        if emls[em][0] != None and emls[em][1] != None:
            HT_dislodge.append(emls[em][0])
            HS_dislodge.append(emls[em][1])
            HS_deposit.append(emls[em][2])
            HT_deposit.append(emls[em][3])
    
    for nl in range(len(nls)):
        if nls[nl][0] != None and nls[nl][1] != None:
            nls_u.append(nls[nl][0])
            nls_transport.append(nls[nl][1])
    
    sliding_n = nls_transport.count("Sliding")
    oroll_n = nls_transport.count("Overturn or Rolling")
    salt_n = nls_transport.count("Saltation")
        
    for gl in range(len(gls)):
        if gls[gl][0] != None and gls[gl][1] != None:
            gls_t.append(gls[gl][0])
            gls_s.append(gls[gl][1])
        
    plt.hist(HT_dislodge, bins=50)
    plt.title("Density plots for Engel and May(miu:{0:.2f})")
    plt.xlabel("Minimum Tsunami Height to Dislodge JBB Boulder")
    plt.ylabel("Count")
    plt.savefig("Density plots(EM)(HT Dislodge)({0:.2f}).png".format(miu))
    plt.clf()

    plt.hist(HS_dislodge, bins=50)
    plt.title("Density plots for Engel and May(miu:{0:.2f})".format(miu))
    plt.xlabel("Minimum Storm Height to Dislodge JBB Boulder")
    plt.ylabel("Count")
    plt.savefig("Density plots(EM)(HS Dislodge)({0:.2f}).png".format(miu))
    plt.clf()

    plt.hist(HS_deposit, bins=50)
    plt.title("Density plots for Engel and May(miu:{0:.2f})".format(miu))
    plt.xlabel("Minimum Storm Height to deposit boulder")
    plt.ylabel("Count")
    plt.savefig("Density plots(EM)(HS Deposit)({0:.2f}).png".format(miu))
    plt.clf()

    plt.hist(HT_deposit, bins=50)
    plt.title("Density plots for Engel and May(miu:{0:.2f})".format(miu))
    plt.xlabel("Minimum Tsunami Height to deposit boulder")
    plt.ylabel("Count")
    plt.savefig("Density plots(EM)(HT Deposit)({0:.2f}).png".format(miu))
    plt.clf()

    # Density Plots for Goto Storms
    plt.hist(gls_s, bins=50)
    plt.title("Density plots for Goto Storms (H:{0:.2f}, Vmiu:{0:.2f})".format(H, v_miu))
    plt.xlabel("Boulder speeds")
    plt.ylabel("Count")
    plt.savefig("Density plots(G)(S)(H{0:.2f}, Vmiu{0:.2f}).png".format(H, v_miu))
    plt.clf()
    
    # Density Plots for Goto Tsunamis
    plt.hist(gls_t, bins=50)
    plt.title("Density plots for Goto Tsunamis (H:{0:.2f}, Vmiu:{0:.2f})".format(H, v_miu))
    plt.xlabel("Boulder speeds")
    plt.ylabel("Count")
    plt.savefig("Density plots(G)(T)(H{0:.2f}, Vmiu{0:.2f}).png".format(H, v_miu))
    plt.clf()

    # Density Plots for Nandasena
    plt.hist(nls_u, bins=50)
    plt.title("Density plots for Nandasena flow velocity")
    plt.xlabel("Minimum flow velocity for incipient motion")
    plt.ylabel("Count")
    plt.savefig("Density plots(N).png")
    plt.clf()
    
    # Density Plots for Nandasena
    plt.bar(["Sliding", "Overturn or Rolling", "Saltation"], [sliding_n, oroll_n, salt_n])
    plt.title("Bar plots for Nandasena transport mode")
    plt.xlabel("Transport Mode")
    plt.ylabel("Count")
    plt.savefig("Bar plots(N).png")
    plt.clf()

    # Generate predicted vs actual for lateral_transport_distance [Pignatelli Model]
    pls_transport_distance = []
    for p in pls:
        pls_transport_distance.append(p[1])

    plt.scatter(pls_transport_distance, pls_actual)
    plt.plot(pls_actual, pls_actual)
    plt.title("Scatter plot for Pignatelli(mann:{0:.2f}, alpha:{1:.2f}, hc:{2:.2f}, Xmax:{3:.2f})".format(manning, alpha, h_c, X_max))
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.savefig("Scatter plot(P)(mann{0:.2f}, alpha{1:.2f}, hc{2:.2f}, Xmax{3:.2f}).png".format(manning, alpha, h_c, X_max))
    plt.clf()

In [8]:
# Fetch data from boulder data set
db = MySQLdb.connect(host="localhost",  # your host
                     user="root",       # username
                     passwd="$u1840974f22603",     # password
                     db="boulders")   # name of the database

# Create a Cursor object to execute queries.
cur = db.cursor()


cur.execute('SELECT*FROM boulder_data4')     # Select data from table using SQL query.
boulder_sql = cur.fetchall()              # Fetch all the remaining rows. Displays as tuples.

boulder_df = pd.DataFrame(boulder_sql)

# Convert data to float.
boulder_df.iloc[:, 6:20] = boulder_df.iloc[:, 6:20].apply(pd.to_numeric)
boulder_df.iloc[:, 22:25] = boulder_df.iloc[:, 22:25].apply(pd.to_numeric)
boulder_df.iloc[:, 27] = boulder_df.iloc[:, 27].apply(pd.to_numeric)
boulder_df.iloc[:, 29] = boulder_df.iloc[:, 29].apply(pd.to_numeric)

#boulder_df.iloc[127,:] # Display data at row and column

In [9]:
# Filter the SQL database

#boulder_df1 = boulder_df.iloc[345:457, 1:41]

boulder_df1 = boulder_df.iloc[:, 1:41]

ls = []

for ind, row in boulder_df1.iterrows():
    # Assign variable names to desired variables
    pre_transport = boulder_df1.loc[ind, 21]
    boulder_density = boulder_df1.loc[ind, 14]
    Vabc = boulder_df1.loc[ind, 9]
    abc = (boulder_df1.loc[ind, 6], boulder_df1.loc[ind, 7], boulder_df1.loc[ind, 8])
    
    if np.isnan(boulder_density)==True or np.isnan(Vabc)==True or np.isnan(abc[0])==True or np.isnan(abc[1])==True or np.isnan(abc[2])==True:
        ls.append(ind)

filt_df = boulder_df1.drop(ls)

In [10]:
# Pick a set of parameters

# Parameters from Engel and May, Goto, Pignatelli, Nandasena
water_density = 1020
c_l = 0.178 # lift coefficient

# Paramter from Engel and May
miu = 0.65 # coefficient of static friction. estimated by Benner

# Pignatelli, Nandasena also requires alpha
manning = 0.03
alpha = 35
alpha = (alpha/360)*(2*np.pi) # Convert to radians
h_c = 0
X_max = 190                   

# Parameter for Nandasena
miu_s = 0.7

# Parameters from Goto
c_d = 1.05 # drag coefficient estimated from Goto
c_m = 1.67 # mass coefficient estimated from Goto # Nandasena also took 2.0
g = 9.81
H = 6 # Tsunami height
v_miu = 0.65 # variable miu 

In [1]:
# Fit the Pignatelli model

'''
# Vary the Manning Number
rms_ls = []
mann_ls = []
for i in manning_dict:     # iterate through manning table
    ran = np.arange(manning_dict[i][0], manning_dict[i][1]+0.0005, 0.005)  # generate range of possible manning values per coastal type
    for mann in ran: # filter output data into those with manning number of interest
        manning = mann
        results = runmodels() # emls, gls, nls, pls, lt_actual
        rms_value = rms(results[4], results[5])
        rms_ls.append(rms_value)
        mann_ls.append(manning)

plt.plot(mann_ls, rms_ls)
plt.title("RMS against Manning")
plt.xlabel("Manning Number")
plt.ylabel("RMS")
plt.show()


# Vary alpha
rmsalpha_ls = []
alpha_ls = []

for a in range(0, 90, 2):
    alpha = a 
    alpha = (alpha/360)*(2*np.pi)
    results = runmodels()
    rms_value = rms(results[4], results[5])
    rmsalpha_ls.append(rms_value)
    alpha_ls.append(a)

plt.plot(alpha_ls, rmsalpha_ls)
plt.title("RMS against Bed Slope")
plt.xlabel("Bed Slope")
plt.ylabel("RMS")
plt.show()


# Vary h_c
rmshc_ls = []
hc_ls = []

for h in np.arange(0, 5, 0.1):
    h_c =  h
    results = runmodels()
    rms_value = rms(results[4], results[5])
    rmshc_ls.append(rms_value)
    hc_ls.append(h_c)

plt.plot(hc_ls, rmshc_ls)
plt.title("RMS against Cliff Height")
plt.xlabel("Cliff Height")
plt.ylabel("RMS")
plt.show()

# Vary X_max
rmsx_ls = []
x_ls = []

for x in np.arange(100, 1000, 10):
    X_max = x
    results = runmodels()
    rms_value = rms(results[4], results[5])
    rmsx_ls.append(rms_value)
    x_ls.append(x)

plt.plot(x_ls, rmsx_ls)
plt.title("RMS against Inundation Distance")
plt.xlabel("Inundation Distance")
plt.ylabel("RMS")
plt.show()
'''

'\n# Vary the Manning Number\nrms_ls = []\nmann_ls = []\nfor i in manning_dict:     # iterate through manning table\n    ran = np.arange(manning_dict[i][0], manning_dict[i][1]+0.0005, 0.005)  # generate range of possible manning values per coastal type\n    for mann in ran: # filter output data into those with manning number of interest\n        manning = mann\n        results = runmodels() # emls, gls, nls, pls, lt_actual\n        rms_value = rms(results[4], results[5])\n        rms_ls.append(rms_value)\n        mann_ls.append(manning)\n\nplt.plot(mann_ls, rms_ls)\nplt.title("RMS against Manning")\nplt.xlabel("Manning Number")\nplt.ylabel("RMS")\nplt.show()\n\n\n# Vary alpha\nrmsalpha_ls = []\nalpha_ls = []\n\nfor a in range(0, 90, 2):\n    alpha = a \n    alpha = (alpha/360)*(2*np.pi)\n    results = runmodels()\n    rms_value = rms(results[4], results[5])\n    rmsalpha_ls.append(rms_value)\n    alpha_ls.append(a)\n\nplt.plot(alpha_ls, rmsalpha_ls)\nplt.title("RMS against Bed Slope")\

In [36]:
resultsplots = runmodels() # return(emls, gls, nls, pls, lt_actual)
interestplots(resultsplots[0], resultsplots[1], resultsplots[2], resultsplots[3], resultsplots[4])

<Figure size 432x288 with 0 Axes>